# 4 LLM 实战演练 1

导入模型和 IMDB 数据集。

In [2]:
from transformers import pipeline

model_id = 'unsloth/gemma-2-2b-it-bnb-4bit'

model = pipeline('text-generation', model_id)

/home/alumkal/.conda/envs/py310-torch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/alumkal/.conda/envs/py310-torch/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [3]:
from IPython.display import display, Markdown

prompt = 'Introduce yourself.'
response = model([dict(role='user', content=prompt)], max_new_tokens=4096)
display(Markdown(response[0]['generated_text'][-1]['content']))

/home/alumkal/.conda/envs/py310-torch/lib/python3.10/site-packages/transformers/generation/utils.py:1338: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Hello! 👋  I'm Gemma, an AI assistant created by the Gemma team.  I'm here to help you with any questions or tasks you might have.  

I'm a large language model, which means I'm really good at understanding and generating text.  I can do things like:

* **Answer your questions:**  I've been trained on a massive amount of text data, so I can provide information on a wide range of topics.
* **Write different kinds of creative text formats:**  From poems to code, I can help you with your writing needs.
* **Summarize text:**  Give me a long piece of writing, and I can give you a concise summary.

What can I do for you today? 😊 


In [4]:
from datasets import load_dataset

imdb = load_dataset('stanfordnlp/imdb')
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

以下 prompt 由作者人工撰写，未经调优。Few-shot learning 数据取自训练集。

In [5]:
def generate_prompt(text, shot=0):
    prompt = 'Please identify the sentiment of the following movie review. Output a single word, either "Positive" or "Negative", as your answer.\n\n'
    examples = [
        f"Example 1: {imdb['train'][0]['text']}\n\nAnswer: Negative\n\n",
        f"Example 2: {imdb['train'][12500]['text']}\n\nAnswer: Positive\n\n",
        f"Example 3: {imdb['train'][12499]['text']}\n\nAnswer: Negative\n\n",
        f"Example 4: {imdb['train'][24999]['text']}\n\nAnswer: Positive\n\n",
    ]
    prompt += ''.join(examples[:shot])
    prompt += f'Movie review: {text}\n\nAnswer: '
    return prompt

prompt = generate_prompt(imdb['test'][-1]['text'], shot=2)
display(Markdown(prompt))

Please identify the sentiment of the following movie review. Output a single word, either "Positive" or "Negative", as your answer.

Example 1: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot.

Answer: Negative

Example 2: Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.

Answer: Positive

Movie review: I caught this movie on the Sci-Fi channel recently. It actually turned out to be pretty decent as far as B-list horror/suspense films go. Two guys (one naive and one loud mouthed a**) take a road trip to stop a wedding but have the worst possible luck when a maniac in a freaky, make-shift tank/truck hybrid decides to play cat-and-mouse with them. Things are further complicated when they pick up a ridiculously whorish hitchhiker. What makes this film unique is that the combination of comedy and terror actually work in this movie, unlike so many others. The two guys are likable enough and there are some good chase/suspense scenes. Nice pacing and comic timing make this movie more than passable for the horror/slasher buff. Definitely worth checking out.

Answer: 

在 IMDB 数据集中取前 100 个负样本和前 100 个正样本进行测试。若输出的第一个 token 不是 `Positive` 或 `Negative`，视作无效回答。

In [6]:
text = imdb['test'][:100]['text'] + imdb['test'][12500:12600]['text']
label = imdb['test'][:100]['label'] + imdb['test'][12500:12600]['label']

In [7]:
for shot in [0, 2, 4]:
    correct, incorrect, invalid = 0, 0, 0
    for i in range(len(text)):
        prompt = generate_prompt(text[i], shot=shot)
        response = model([dict(role='user', content=prompt)], max_new_tokens=1)
        response = response[0]['generated_text'][-1]['content']
        if response not in ['Positive', 'Negative']:
            invalid += 1
        elif (response == 'Positive') == label[i]:
            correct += 1
        else:
            incorrect += 1
    print(f'{shot}-shot: {(correct + invalid / 2) / len(text) * 100:.2f}%, {correct=}, {incorrect=}, {invalid=}')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0-shot: 94.50%, correct=186, incorrect=8, invalid=6
2-shot: 96.50%, correct=193, incorrect=7, invalid=0
4-shot: 95.00%, correct=190, incorrect=10, invalid=0


可以看出，few-shot learning 对模型准确率提升不大，推测限制准确率的主要因素为模型本身的语言理解能力。

值得注意的是，在给出示例后，模型生成的所有回答都是有效的。这说明 few-shot learning 对模型遵守固定格式的能力提升很明显。